In [1]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score,mean_absolute_error,median_absolute_error,explained_variance_score
from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV,KFold,StratifiedKFold,RandomizedSearchCV #交叉验证
from sklearn.preprocessing import StandardScaler #特征标准化
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.inspection import PartialDependenceDisplay #部分依赖图
import warnings
warnings.filterwarnings("ignore")
from sklearn.model_selection import train_test_split  # 划分训练集、验证集、测试集
from sklearn.svm import SVR #支持向量机
from sklearn.neural_network import MLPRegressor #神经网络
from sklearn.tree import DecisionTreeRegressor #决策树
from sklearn.svm import SVR #支持向量机
# from xgboost.sklearn import XGBRegressor

from sklearn.linear_model import LogisticRegression #Logit
from sklearn.tree import DecisionTreeClassifier #决策树
from sklearn.svm import SVC #支持向量机
from sklearn.ensemble import RandomForestClassifier #随机森林
from sklearn.ensemble import GradientBoostingClassifier #梯度提升树
# from xgboost.sklearn import XGBClassifier #XGBoost
from sklearn.neural_network import MLPClassifier #神经网络
from sklearn.model_selection import GridSearchCV,StratifiedKFold,RandomizedSearchCV #交叉验证
from sklearn.model_selection import train_test_split  # 划分训练集、验证集、测试集
from sklearn.preprocessing import StandardScaler #特征标准化
from sklearn.metrics import roc_curve, auc,RocCurveDisplay  # ROC曲线，计算AUC
from sklearn.inspection import PartialDependenceDisplay #部分依赖图
from sklearn import metrics
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import PrecisionRecallDisplay
from sklearn.metrics import average_precision_score
from sklearn.metrics import mean_squared_error, r2_score,mean_absolute_error,median_absolute_error,explained_variance_score
import time
import random
from sklearn.metrics import fbeta_scorefrom pathlib import Path


In [ ]:
TARGET_FOLDER = '参考文献/1/20240618102625WU_FILE_1'

def locate_project_root(target_folder=TARGET_FOLDER):
    current = Path.cwd().resolve()
    for candidate in [current, *current.parents]:
        if (candidate / target_folder).exists():
            return candidate
    raise FileNotFoundError(f'未能在 {current} 及其父目录中定位 {target_folder}')

PROJECT_ROOT = locate_project_root()
DATA_DIR = PROJECT_ROOT / TARGET_FOLDER / '数据' / '数据-python'
OUTPUT_DIR = PROJECT_ROOT / 'output'
TABLE_DIR = OUTPUT_DIR / 'tables'
FIG_DIR = OUTPUT_DIR / 'figures'
ML_DIR = OUTPUT_DIR / 'ml'
for path in (TABLE_DIR, FIG_DIR, ML_DIR):
    path.mkdir(parents=True, exist_ok=True)
print(f'PROJECT_ROOT: {PROJECT_ROOT}')


In [ ]:
###### 数据导入
data = pd.read_csv(DATA_DIR / 'data.csv', header=0)
data = pd.DataFrame(data)
print(data.head(3))
print(data.shape)

In [ ]:
###### 数据预处理
x = data.iloc[:, 6:]
y = data.iloc[:, 5] #是否发放股利

x_train1 = x.loc[data['year']==2006]
y_train1 = y.loc[data['year']==2006]
sc = StandardScaler()
sc.fit(x_train1)
x_train1 = sc.transform(x_train1)
x_train1 = pd.DataFrame(x_train1,columns= x.columns)

for i in range(2,18):
    exec ("x_train%s=1"%i)
    exec ("y_train%s=1"%i)

x_train = [x_train1,x_train2,x_train3,x_train4,x_train5,x_train6,x_train7,x_train8,x_train9,x_train10,x_train11,x_train12,x_train13,
           x_train14,x_train15,x_train16,x_train17]
y_train = [y_train1,y_train2,y_train3,y_train4,y_train5,y_train6,y_train7,y_train8,y_train9,y_train10,y_train11,y_train12,y_train13,
          y_train14,y_train15,y_train16,y_train17]

for i in range(1,18):
    j = i + 2005
    k = i - 1
    x_train[k] = x.loc[data['year']== j]
    y_train[k] = y.loc[data['year']== j]
    x_train[k] = sc.transform(x_train[k])
    x_train[k] = pd.DataFrame(x_train[k],columns= x.columns)

In [ ]:
x_test = sc.transform(x)
x_test = pd.DataFrame(x_test,columns= x.columns)

names = list(x_train1.columns)

In [ ]:
names_chinese = [ '管理费用率', '管理层持股比例', '独立董事比例','董事会女性比例', '董事长持股比例', '董事长年龄',
                 '董事长任期','董事长薪酬', '股权激励虚拟变量',  '其他应收款资产比', '股权集中度','股权制衡度','中小股东持股比例', 
                 '机构投资者持股比例', '控股股东股权质押比例', '财务报告质量','留存收益资产比','自由现金流', 
                 '税收规避程度', '实际税率', '纳税波动率','融资约束程度', '再融资动机', '上一期股利水平','资产收益率',
                 '每股经营活动现金流量', '托宾Q', '账面市值比', '资产负债率', '产权性质',
                 '销售增长率', '公司规模','分析师跟踪人数','公司所在省份市场化程度','ind1', 'ind2', 'ind3' ,'ind4' ,'ind4' ,'ind5',
                 'ind7', 'ind8' ,'ind11' ,'ind12' ,'ind15' ,'ind16' ,'ind17' ,'ind18' ,'ind19' ,'ind20' ,'ind21',
                 'ind22', 'ind23', 'ind24', 'ind25' ,'ind26' ,'ind27' ,'ind28' ,'ind29' ,'ind30' ,'ind31' ,'ind32',
                 'ind33' ,'ind34' ,'ind35' ,'ind37' ,'ind38' ,'ind39' ,'ind40' ,'ind41' ,'ind42']

In [ ]:
result = []
result1 = []
result2 = []
result3 = [] 
result4 = []
result5 = []
result6 = [] 
# result7 = []
# result8 = []
for i in range(0,1):
    j = i+1
    gbc = GradientBoostingClassifier(n_estimators =1000 , max_depth = 2,subsample = 0.2,learning_rate = 0.01,random_state=0)
    result_gbc = gbc.fit(x_train[i],y_train[i])
    pred_gbc = gbc.predict(x_train[j])
    pred_gbc1 = gbc.predict(x_train[i])
    pred_pro_gbc = gbc.predict_proba(x_train[j])
    fpr_gbc, tpr_gbc, thresholds_gbc = roc_curve(y_train[j], pred_pro_gbc[:, 1])
    roc_auc_gbc = auc(fpr_gbc, tpr_gbc) #auc值
    result.append(roc_auc_gbc)
    average_precision_gbc = average_precision_score(y_train[j], pred_pro_gbc[:, 1])
    result1.append(average_precision_gbc)
    r2 = gbc.score(x_train[i],y_train[i], sample_weight=None)
    result2.append(r2)
    r2a = gbc.score(x_train[j],y_train[j], sample_weight=None)
    result3.append(r2)
    mse_predict = mean_squared_error(y_train[j],pred_gbc)
    result4.append(mse_predict)
    mse_predict1 = mean_squared_error(y_train[i],pred_gbc1)
    result6.append(mse_predict1)
    mae_predict = mean_absolute_error(y_train[j],pred_gbc)
    result5.append(mae_predict)
print('AUC = ', '%.4f'%np.mean(result)) 
print('AP = ', '%.4f'%np.mean(result1)) 
print('样本内R方=','%.4f'%np.mean(result2))
print('样本外R方=','%.4f'%np.mean(result3))
print('MSE=','%.4f'%np.mean(result4))
print('MAE=','%.4f'%np.mean(result5))
print('MSEnei=','%.4f'%np.mean(result6))

In [ ]:
model_gbr_fig = GradientBoostingClassifier(n_estimators =1000 , max_depth = 2,subsample = 0.2,learning_rate = 0.01,random_state=0)
model_gbr_fig.fit(x_test,y)

In [ ]:
plt.rcParams['font.sans-serif']=['SimHei']
plt.rcParams['axes.unicode_minus']=False #用来正常显示负号
PartialDependenceDisplay.from_estimator(model_gbr_fig,x_test,['Analyst_num'],grid_resolution=100,n_jobs = -1,method = 'brute')
plt.ylabel('股利支付意愿',fontsize = 13)
plt.xlabel('分析师跟踪人数',fontsize = 13)
plt.title('渐进梯度分类树',fontsize = 13)
plt.tick_params(labelsize=13)
plt.subplots_adjust(left=0.15,bottom = 0.15)
ax=plt.gca() 
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['bottom'].set_visible(False)
ax.spines['left'].set_visible(False)
ax.yaxis.grid(linewidth=0.1,color='black',linestyle='--')
ax.tick_params(bottom=False,top=False,left=False,right=False)
plt.savefig(ML_DIR / '分析师跟踪人数.png', dpi=200)

In [ ]:
plt.rcParams['font.sans-serif']=['SimHei']
plt.rcParams['axes.unicode_minus']=False #用来正常显示负号
PartialDependenceDisplay.from_estimator(model_gbr_fig,x_test,['Cashflow'],grid_resolution=100,n_jobs = -1,method = 'brute')
plt.ylabel('股利支付意愿',fontsize = 13)
plt.xlabel('每股经营活动现金流量',fontsize = 13)
plt.title('渐进梯度分类树',fontsize = 13)
plt.tick_params(labelsize=13)
plt.subplots_adjust(left=0.15,bottom = 0.15)
ax=plt.gca() 
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['bottom'].set_visible(False)
ax.spines['left'].set_visible(False)
ax.yaxis.grid(linewidth=0.1,color='black',linestyle='--')
ax.tick_params(bottom=False,top=False,left=False,right=False)
plt.savefig(ML_DIR / '每股经营活动现金流量.png', dpi=200)

In [ ]:
result = []
result1 = []
result2 = []
result3 = [] 
result4 = []
result5 = []
# result6 = [] 
# result7 = []
# result8 = []
for i in range(0,16):
    j = i+1
    forest = RandomForestClassifier(n_estimators=5000, max_features=12,random_state=0, n_jobs=-1)
    result_forest = forest.fit(x_train[i],y_train[i])
    pred_forest = forest.predict(x_train[j])
    pred_forest1 = forest.predict(x_train[i])
    pred_pro_forest = forest.predict_proba(x_train[j])
    fpr_forest, tpr_forest, thresholds_forest = roc_curve(y_train[j], pred_pro_forest[:, 1])
    roc_auc_forest = auc(fpr_forest, tpr_forest) #auc值
    result.append(roc_auc_forest)
    average_precision_forest = average_precision_score(y_train[j], pred_pro_forest[:, 1])
    result1.append(average_precision_forest)
    r2 = forest.score(x_train[i],y_train[i], sample_weight=None)
    result2.append(r2)
    r2a = forest.score(x_train[j],y_train[j], sample_weight=None)
    result3.append(r2)
    mse_predict = mean_squared_error(y_train[j],pred_forest)
    result4.append(mse_predict)
    mse_predict1 = mean_squared_error(y_train[i],pred_forest1)
    result6.append(mse_predict1)
    mae_predict = mean_absolute_error(y_train[j],pred_forest)
    result5.append(mae_predict)
print('AUC = ', '%.4f'%np.mean(result)) 
print('AP = ', '%.4f'%np.mean(result1)) 
print('样本内R方=','%.4f'%np.mean(result2))
print('样本外R方=','%.4f'%np.mean(result3))
print('MSE=','%.4f'%np.mean(result4))
print('MAE=','%.4f'%np.mean(result5))
print('MSEnei=','%.4f'%np.mean(result6))

In [ ]:
model_forest_fig = RandomForestClassifier(n_estimators=5000, max_features=12,random_state=0, n_jobs=-1)
model_forest_fig.fit(x_test,y)
sorted_index = model_forest_fig.feature_importances_.argsort()
print(sorted_index)
for i in range(x.shape[1]):
    print(x.columns[sorted_index[i]])
for i in range(x.shape[1]):
    print(forest.feature_importances_[sorted_index[i]])

In [ ]:
plt.rcParams['font.sans-serif']=['SimHei']
plt.rcParams['axes.unicode_minus']=False #用来正常显示负号
PartialDependenceDisplay.from_estimator(model_forest_fig,x_test,['Cashflow'],grid_resolution=100,method = 'brute')
plt.ylabel('股利支付意愿',fontsize = 13)
plt.xlabel('每股经营活动现金流量',fontsize = 13)
plt.title('随机森林',fontsize = 13)
plt.tick_params(labelsize=13)
plt.subplots_adjust(left=0.15,bottom = 0.15)
ax=plt.gca() 
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['bottom'].set_visible(False)
ax.spines['left'].set_visible(False)
ax.yaxis.grid(linewidth=0.1,color='black',linestyle='--')
ax.tick_params(bottom=False,top=False,left=False,right=False)
plt.savefig(ML_DIR / '每股经营活动现金流量1.png', dpi=200)

In [ ]:
plt.rcParams['font.sans-serif']=['SimHei']
plt.rcParams['axes.unicode_minus']=False #用来正常显示负号
PartialDependenceDisplay.from_estimator(model_forest_fig,x_test,['Analyst_num'],grid_resolution=100,method = 'brute')
plt.ylabel('股利支付意愿',fontsize = 13)
plt.xlabel('分析师跟踪人数',fontsize = 13)
plt.title('随机森林',fontsize = 13)
plt.tick_params(labelsize=13)
plt.subplots_adjust(left=0.15,bottom = 0.15)
ax=plt.gca() 
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['bottom'].set_visible(False)
ax.spines['left'].set_visible(False)
ax.yaxis.grid(linewidth=0.1,color='black',linestyle='--')
ax.tick_params(bottom=False,top=False,left=False,right=False)
plt.savefig(ML_DIR / '分析师跟踪人数1.png', dpi=200)